## ODC with Zarr, Kerchunk, and earthaccess

In [ ]:
import earthaccess
import fsspec
import xarray as xr
from odc.geo.geobox import GeoBox
from odc.geo.xr import xr_reproject

In [ ]:
def warp_resample():
    src = "earthaccess_data/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json"
    variable = "analysed_sst"
    dstSRS = "EPSG:3857"
    srcSRS = "EPSG:4326"
    width = height = 256
    te = [
        -20037508.342789244,
        -20037508.342789244,
        20037508.342789244,
        20037508.342789244,
    ]
    earthaccess.login()
    s3_fs = earthaccess.get_s3fs_session(daac="PODAAC")
    storage_options = s3_fs.storage_options.copy()
    fsspec_caching = {
        "cache_type": "none",
    }
    fs = fsspec.filesystem("reference", fo=src, **fsspec_caching)
    m = fs.get_mapper("")
    da = xr.open_dataset(
        m, engine="kerchunk", chunks={}, storage_options=storage_options
    )[variable]
    gbox = GeoBox.from_bbox(te, dstSRS, shape=(height, width))
    da = da.odc.assign_crs(srcSRS)
    return xr_reproject(da, gbox)

In [ ]:
if __name__ == "__main__":
    da, gbox = warp_resample()